# Parameter tuning for Classification and Regression Tree (RF) classification

### Testing described parameter settings for RF algorithm using Google Earth Engine Python API and NICFI Normalized Analytic Basemap from December 2022

Author: Finn Geiger\
Date: March 30th 2023\
Contact:
- https://github.com/finn-geiger
- https://www.linkedin.com/in/finn-geiger-b1329a20b/

### 1 Import and setup
#### 1.1 Importing the required libraries and packages

In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geemap
import ee
import os
import time
import pandas as pd
from tabulate import tabulate
#%pip install tabulate


The following classes and landcover IDs will be used:

In [6]:
info = {'Class name': ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies'],
        'landcover ID': [1, 2, 3, 4, 5, 6, 7]}

print(tabulate(info, headers='keys', tablefmt='fancy_grid'))

╒══════════════╤════════════════╕
│ Class name   │   landcover ID │
╞══════════════╪════════════════╡
│ Informal     │              1 │
├──────────────┼────────────────┤
│ Formal       │              2 │
├──────────────┼────────────────┤
│ Industrial   │              3 │
├──────────────┼────────────────┤
│ Roads        │              4 │
├──────────────┼────────────────┤
│ Vacant land  │              5 │
├──────────────┼────────────────┤
│ Vegetation   │              6 │
├──────────────┼────────────────┤
│ Water-bodies │              7 │
╘══════════════╧════════════════╛


##### When first using the GEE Python API the user must authenticate and initialize the environment by using the following two lines of codes:

In [7]:
#ee.Authenticate() 
#ee.Initialize()

In [8]:
# creating the map
Map = geemap.Map()

# loading the interactive map
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

#### 1.2 Importing the datasets from GEE assets and data catalog and clipping the basemap to the AOI

In [9]:
# Loading the Base scene
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')

# Filter basemaps by date and get the first image from filtered results
basemap_2022_12 = nicfi.filter(ee.Filter.date('2022-12-01','2023-01-01')).first()

# Visualizing the scene
vis_params = {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}

# Adding the basemap to the map
Map.centerObject(basemap_2022_12, 4)
Map.addLayer(basemap_2022_12, vis_params, '2022-12 mosaic')

In [10]:
# Loading the AOI and Masking the base scene
vis_params_aoi = {'color': 'blue'}
aoi_windhoek = ee.FeatureCollection('users/s85315/masterthesis/Study_Area_Windhoek')

# Adding the AOI to the map
Map.addLayer(aoi_windhoek, vis_params_aoi, 'AOI')
Map.centerObject(aoi_windhoek, 12)

In [11]:
# clipping the basescene to the AOI
basescene = basemap_2022_12.clipToCollection(aoi_windhoek)
Map.addLayer(basescene, vis_params, 'clipped')

In [12]:
# importing training data samples and adding them to the map
# Transforming the Geometries into FeatureCollections and applying properties
TS_Informal_Points = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/TS_Informal_RPoints')
TS_Formal_Points = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/TS_Formal_RPoints')
TS_Industrial_Points = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/TS_Industrial_RPoints')
TS_Roads_Points = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/TS_Roads_RPoints')
TS_VacantLand_Points = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/TS_VacantLand_RPoints')
TS_Vegetation_Points = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/TS_Vegetation_RPoints')
TS_Waterbodies_Points = ee.FeatureCollection('users/s85315/masterthesis/TrainingSamples/TS_Waterbodies_RPoints')

# adding the samples to the map
# Map.addLayer(TS_Informal_Points, {'color': 'c43c39'}, 'Informal Training Data', False)
# Map.addLayer(TS_Formal_Points, {'color': 'e5b636'}, 'Formal Training Data', False)
# Map.addLayer(TS_Industrial_Points, {'color': '2f2f2f'}, 'Industrial Training Data', False)
# Map.addLayer(TS_Roads_Points, {'color': 'aaaaaa'}, 'Roads Training Data', False)
# Map.addLayer(TS_VacantLand_Points, {'color': 'b08e7a'}, 'Vacant land Training Data', False)
# Map.addLayer(TS_Vegetation_Points, {'color': '85b66f'}, 'Vegetation Training Data', False)
# Map.addLayer(TS_Waterbodies_Points, {'color': 'a5bfdd'}, 'Waterbodies Training Data', False)

# merging all FeatureCollections into one layer
training_samples = TS_Informal_Points.merge(TS_Formal_Points).merge(TS_Industrial_Points).merge(TS_Roads_Points).merge(TS_VacantLand_Points).merge(TS_Vegetation_Points).merge(TS_Waterbodies_Points)

### 2 Classification with RF

#### 2.1 Applying training samples on the base scene

In [13]:
# adding the training samples to the basescene
training = basescene.sampleRegions(**{
    'collection': training_samples, 
    'properties': ['landcover'], 
    'scale': 4.77
})

#### 2.2 Configuration and creation of the empty RF classifier

In [497]:
# creating variable for parameter settings
# Mtry will be set to default
# Ntree values between [50:550] will be tested using a step size of 50
RF_param = 550
max_nodes_from_pt_cart = 40
classifier_params = {
              'numberOfTrees':RF_param, # 	Ntree; The number of decision trees to create.
              'variablesPerSplit':None, # Mtry; The number of variables per split. If unspecified, uses the square root of the number of variables.
              'minLeafPopulation':1, # smallest sample size possible per leaf
              'bagFraction':0.5, #The fraction of input to bag per tree.
              'maxNodes':None, # the number of individual decision tree models
              #'maxNodes':max_nodes_from_pt_cart, # the number of individual decision tree models
              'seed': 0}  # The randomization seed.


# creating the classifier using RF
classifier = ee.Classifier.smileRandomForest(**classifier_params).train(**{
  'features': training,  
  'classProperty': 'landcover', 
  'inputProperties': basescene.bandNames()
})

#### 2.3 classifying the basescene and visualizing the product

In [498]:
# classifying the basescene using the created classifier
classified_basescene = basescene.classify(classifier)

# creating the visualization parameters
palette = ['c43c39', 'e5b636', '2f2f2f', 'aaaaaa', 'b08e7a', '85b66f', 'a5bfdd']
vis_params_classified = {'min': 1, 'max': 7, 'palette': palette}


Map.addLayer(classified_basescene, vis_params_classified, 'classified basescene')


#### 2.4 Exporting the results

##### 2.4.1 Exporting to Google Drive

In [499]:
# converting the FeatureCollection to Geometry for export
aoi_geom = aoi_windhoek.geometry()

# exporting to Google drive with GEE API
# export = ee.batch.Export.image.toDrive(**{
#     'image': classified_basescene,
#     'description': 'classified_map', # TODO: change name here
#     'folder': 'data', # TODO: change name here
#     'scale': 4.77,
#     'region': aoi_geom
# })

# # starting the process
# export.start()

# # tracking the process
# while export.active():
#   print('Polling for task (id: {}).'.format(export.id))
#   time.sleep(5)

##### 2.4.2 Exporting to Asset

In [500]:
# exporting to Google Asset
# export = ee.batch.Export.image.toAsset(**{
#   'image': classified_basescene,
#   'description': 'Export classified map',
#   'assetId': 'users/s85315/masterthesis/Testing/export_basescene', # TODO: change name here
#   'scale': 100,
#   'region': aoi_geom
# })

# # starting the process
# export.start()

# # tracking the process
# while export.active():
#   print('Polling for task (id: {}).'.format(export.id))
#   time.sleep(5)

### 3 Accuracy assessment

#### 3.1 Importing validation samples from GEE Assets

In [501]:
# Transforming the Geometries into FeatureCollections and applying properties
VS_Informal_Points = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/VS_Informal_RPoints')
VS_Formal_Points = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/VS_Formal_RPoints')
VS_Industrial_Points = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/VS_Industrial_RPoints')
VS_Roads_Points = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/VS_Roads_RPoints')
VS_VacantLand_Points = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/VS_VacantLand_RPoints')
VS_Vegetation_Points = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/VS_Vegetation_RPoints')
VS_Waterbodies_Points = ee.FeatureCollection('users/s85315/masterthesis/ValidationSamples/VS_Waterbodies_RPoints')

validation_samples = VS_Informal_Points.merge(VS_Formal_Points).merge(VS_Industrial_Points).merge(VS_Roads_Points).merge(VS_VacantLand_Points).merge(VS_Vegetation_Points).merge(VS_Waterbodies_Points)


#### 3.2 Applying the validation samples to the basescene

In [502]:
# applying the validation samples to the classified map
validation = classified_basescene.sampleRegions(**{
  'collection': validation_samples,
  'properties': ['landcover'],
  'tileScale': 16,
  'scale': 4.77,
})

#### 3.3 Generating the error matrix and printing information

In [503]:
basescene_error_matrix = validation.errorMatrix('landcover', 'classification')

# printing statistics
print('Confusion Matrix', basescene_error_matrix.getInfo())
print('Overall Accuracy', basescene_error_matrix.accuracy().getInfo())
print('Producers Accuracy', basescene_error_matrix.producersAccuracy().getInfo())
print('Consumers Accuracy', basescene_error_matrix.consumersAccuracy().getInfo())
print('Kappa', basescene_error_matrix.kappa().getInfo())

Confusion Matrix [[0, 0, 0, 0, 0, 0, 0, 0], [0, 18, 6, 1, 4, 1, 0, 0], [0, 2, 25, 2, 1, 0, 0, 0], [0, 4, 8, 15, 3, 0, 0, 0], [0, 1, 1, 1, 26, 1, 0, 0], [0, 3, 0, 1, 2, 23, 1, 0], [0, 0, 0, 0, 0, 0, 25, 5], [0, 0, 0, 0, 0, 0, 1, 29]]
Overall Accuracy 0.7666666666666667
Producers Accuracy [[0], [0.6], [0.8333333333333334], [0.5], [0.8666666666666667], [0.7666666666666667], [0.8333333333333334], [0.9666666666666667]]
Consumers Accuracy [[0, 0.6428571428571429, 0.625, 0.75, 0.7222222222222222, 0.92, 0.9259259259259259, 0.8529411764705882]]
Kappa 0.7277777777777777


##### 3.3.1 Visualizing the error matrix

In [504]:
# creating a Pandas Dataframe for the error matrix
error_matrix = basescene_error_matrix.getInfo()
df_error_matrix = pd.DataFrame(error_matrix)

# deleting the first row and column since GEE add's a class with the landcover ID 0 by default.
df_error_matrix.columns = ['not used','Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_error_matrix = df_error_matrix.drop(df_error_matrix.columns[0],axis=1)
df_error_matrix.drop(index=df_error_matrix.index[0], axis=0, inplace=True)

# calculating row and column sum of points
column_total = df_error_matrix.sum()
column_total.name = 'Total'
df_error_matrix.loc[8] = column_total
df_error_matrix['Total'] = df_error_matrix.sum(axis=1)

header_error_matrix = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies', 'Total']
df_error_matrix['Names'] = header_error_matrix
df_error_matrix = df_error_matrix.set_index('Names')


print(tabulate(df_error_matrix, headers=header_error_matrix, tablefmt='fancy_grid', showindex=header_error_matrix))
#df_error_matrix.to_csv(f"./accuracies/RF/RF_Error_Matrix_{RF_param}.csv", sep=';', index=True)

╒══════════════╤════════════╤══════════╤══════════════╤═════════╤═══════════════╤══════════════╤════════════════╤═════════╕
│              │   Informal │   Formal │   Industrial │   Roads │   Vacant land │   Vegetation │   Water-bodies │   Total │
╞══════════════╪════════════╪══════════╪══════════════╪═════════╪═══════════════╪══════════════╪════════════════╪═════════╡
│ Informal     │         18 │        6 │            1 │       4 │             1 │            0 │              0 │      30 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Formal       │          2 │       25 │            2 │       1 │             0 │            0 │              0 │      30 │
├──────────────┼────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼────────────────┼─────────┤
│ Industrial   │          4 │        8 │           15 │       3 │             0 │            0 │              0 │      30 │
├───────

##### 3.3.2 Producer's and consumer's accuracy

In [505]:
# creating the lists 
producers = basescene_error_matrix.producersAccuracy().getInfo()
df_producers = pd.DataFrame(producers)
df_producers.drop(index=df_producers.index[0], axis=0, inplace=True)

class_names = ['Informal', 'Formal', 'Industrial', 'Roads', 'Vacant land', 'Vegetation', 'Water-bodies']
df_producers['class names'] = class_names
df_producers.columns = ["Producer Accuracy", "Class name"]
df_producers['Producer Accuracy'] = df_producers['Producer Accuracy'].multiply(100).round(2)


print(tabulate(df_producers, headers=["Producer's Accuracy [%]", "Class name"], tablefmt='fancy_grid',  showindex=False))
#df_producers.to_csv(f"./accuracies/RF/RF_Producers_Accuracy_{RF_param}.csv", sep=';', index=False)

╒═══════════════════════════╤══════════════╕
│   Producer's Accuracy [%] │ Class name   │
╞═══════════════════════════╪══════════════╡
│                     60    │ Informal     │
├───────────────────────────┼──────────────┤
│                     83.33 │ Formal       │
├───────────────────────────┼──────────────┤
│                     50    │ Industrial   │
├───────────────────────────┼──────────────┤
│                     86.67 │ Roads        │
├───────────────────────────┼──────────────┤
│                     76.67 │ Vacant land  │
├───────────────────────────┼──────────────┤
│                     83.33 │ Vegetation   │
├───────────────────────────┼──────────────┤
│                     96.67 │ Water-bodies │
╘═══════════════════════════╧══════════════╛


In [506]:
# creating a dataframe from the list of consumer accuracies and remove landcover ID 0
consumers = basescene_error_matrix.consumersAccuracy().getInfo()
df_consumers = pd.DataFrame(consumers)
df_consumers = df_consumers.drop(df_consumers.columns[0],axis=1)
df_consumers.columns = class_names

# reshaping the dataframe from wide to long format:
df_consumers_long = pd.melt(df_consumers, var_name='Class name', value_name="Consumer Accuracy")
df_consumers_long = df_consumers_long[['Consumer Accuracy', 'Class name']]
df_consumers_long['Consumer Accuracy'] = df_consumers_long['Consumer Accuracy'].multiply(100).round(2)


print(tabulate(df_consumers_long, headers=["Consumers's Accuracy [%]", "Class name"], tablefmt='fancy_grid',  showindex=False))
#df_consumers_long.to_csv(f"./accuracies/RF/RF_Consumers_Accuracy_{RF_param}.csv", sep=';', index=False)

╒════════════════════════════╤══════════════╕
│   Consumers's Accuracy [%] │ Class name   │
╞════════════════════════════╪══════════════╡
│                      64.29 │ Informal     │
├────────────────────────────┼──────────────┤
│                      62.5  │ Formal       │
├────────────────────────────┼──────────────┤
│                      75    │ Industrial   │
├────────────────────────────┼──────────────┤
│                      72.22 │ Roads        │
├────────────────────────────┼──────────────┤
│                      92    │ Vacant land  │
├────────────────────────────┼──────────────┤
│                      92.59 │ Vegetation   │
├────────────────────────────┼──────────────┤
│                      85.29 │ Water-bodies │
╘════════════════════════════╧══════════════╛


##### 3.3.3 Overall Accuracy and Kappa Coefficent

In [507]:
# defining the variables
overall_accuracy = basescene_error_matrix.accuracy().getInfo()
overall_print = str(round(overall_accuracy * 100, 2))
kappa = basescene_error_matrix.kappa().getInfo()

df_overall_kappa = pd.DataFrame()

# printing out vaLues
print("\033[1m" + "Overall Accuracy " + overall_print + " %" + "\033[0m")
print("\033[1m" + "Kappa coefficent " + str(round(kappa, 2)) + "\033[0m")

Overall Accuracy 76.67 %
Kappa coefficent 0.73


##### Resources for code snippets

https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/10_Export.ipynb \
https://worldbank.github.io/OpenNightLights/tutorials/mod6_6_RF_classifier.html \
https://towardsdatascience.com/how-to-easily-create-tables-in-python-2eaea447d8fd